In [2]:
# Import modules
import six
import mule
import numpy as np
import xarray as xr
import os
from datetime import datetime

In [3]:
# Set some config
restart_filepath = '/g/data/p66/rml599/LandUse-tests/pre-industrial-essential.astart'
stash_master_filepath = '/g/data/access/projects/access/umdir/vn10.9/ctldata/STASHmaster/STASHmaster_A'
stash_range = range(851, 860+1)
step2 = True

In [4]:
# Load the file
restart = mule.FieldsFile.from_file(restart_filepath)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/mule/stashmaster.py:259: UserWarning: 
Unable to load STASHmaster from version string, path does not exist
Path: $UMDIR/vn7.3/ctldata/STASHmaster/STASHmaster_A
Please check that the value of mule.stashmaster.STASHMASTER_PATH_PATTERN is correct for your site/configuration
  warnings.warn(msg)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/mule/validators.py:198: UserWarning: 
File: /g/data/p66/rml599/LandUse-tests/pre-industrial-essential.astart
Incorrect dataset_type (found 1, should be one of (3,))
  warnings.warn(msg)


In [5]:
# Extract data from UM file.
def extract_data_array(restart, stash_code, num_levels=5, do_mean=True):

    data = None
    
    for field in restart.fields:
        if field.lbuser4 == stash_code and field.lbuser5 in range(1, num_levels+1):

            # First encounter, create the data object
            if data is None:
                shape = (num_levels, *field.get_data().shape)
                data = np.zeros(shape)

            ix = field.lbuser5 - 1
            data[ix, :, :] = field.get_data()

    # Create a data array
    da = xr.DataArray(
        data,
        dims=('z', 'x', 'y')
    )

    if do_mean is False:
        return da

    # Nan the fill value
    # fill_value = da.min()
    # Leaving this out makes ocean super negative and the data invisible (only use for plotting!!)
    # da = da.where(da != fill_value)
    
    da_mean = da.mean('z')

    return da_mean

In [6]:
# STEP 2 - User predefined vegfrac
vegfrac = '/g/data/p66/rml599/LandUse-tests/vegfrac-dom-tree.nc'
vf = xr.open_dataset(vegfrac)
vfs = vf.fraction.data.squeeze()

In [7]:
# Modify the fields
new_restart = restart.copy()

providers = dict()

# Extract the data
for stash in stash_range:
    new_field = extract_data_array(restart, stash)
    providers[stash] = mule.ArrayDataProvider(new_field.data[:])

# Loop 2 - Create a new field file
for field in restart.fields:

    stash_code = field.lbuser4
    level = field.lbuser5

    # If it is one of the fields we want to modify, do it
    if stash_code in providers.keys() and level <= 5:
        # new_restart.fields.append(operators[stash_code])
        field.set_data_provider(providers[stash_code])

    # STEP 2
    if stash_code in [216, 835] and step2 == True:
        # print(f'Updating {stash_code}')
        dp = mule.ArrayDataProvider(vfs[level-1,:,:])
        field.set_data_provider(dp)
    
    new_restart.fields.append(field)

In [8]:
# Intercept the write function to disable validation
def to_file(self, output_file_or_path):
        """
        Write to an output file or path.

        Args:
            * output_file_or_path (string or file-like):
                An open file or filepath. If a path, it is opened and
                closed again afterwards.

        .. Note::
            As part of this the "validate" method will be called. For the
            base :class:`UMFile` class this does nothing, but sub-classes
            may override it to provide specific validation checks.

        """
        # Call validate - to ensure the file about to be written out doesn't
        # contain obvious errors.  This is done here before any new file is
        # created so that we don't create a blank file if the validation fails
        if isinstance(output_file_or_path, six.string_types):
            self.validate(filename=output_file_or_path, warn=True)
        else:
            self.validate(filename=output_file_or_path.name, warn=True)

        if isinstance(output_file_or_path, six.string_types):
            with open(output_file_or_path, 'wb') as output_file:
                self._write_to_file(output_file)
        else:
            self._write_to_file(output_file_or_path)

new_restart.to_file = to_file

In [9]:
# Write out the data.
stash_start, stash_end = stash_range[0], stash_range[-1]
timestamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
output_filename = f'mean5_{stash_start}-{stash_end}_{timestamp}.astart'
output_filename = f'mean5_VF_{stash_start}-{stash_end}_{timestamp}.astart'
# output_dir = '/g/data/rp23/bjs581/work/rachel'
output_dir = '/g/data/p66/bjs581'
output_filepath = f'{output_dir}/{output_filename}'

new_restart.to_file(new_restart, output_file_or_path=output_filepath)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/mule/validators.py:198: UserWarning: 
File: /g/data/p66/bjs581/mean5_VF_851-860_20231218_230239.astart
Incorrect dataset_type (found 1, should be one of (3,))
  warnings.warn(msg)
